In [179]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

In [180]:
w1 = np.random.rand(128, 28)/100
b1 = np.random.rand(128,28)/100
w2 = np.random.rand(10, 128)/100
b2 = np.random.rand(10,28)/100
w3 = np.random.rand(28, 1)/100

In [181]:
# Load the dataset (grayscale)
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Normalize the images to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

In [182]:
train_images[0].shape

(28, 28)

In [183]:
def model(input):
    output = w1 @ input + b1
    output = np.maximum(0, output)
    output = w2 @ output + b2
    output = np.maximum(0,output)
    output = output @ w3
    output = np.maximum(0,output)
    return output

In [184]:
model(train_images[3])

array([[0.00239579],
       [0.00232259],
       [0.00245733],
       [0.00237833],
       [0.0026729 ],
       [0.00243549],
       [0.00252914],
       [0.00221637],
       [0.00227516],
       [0.00250544]])

In [185]:
def loss_function(predicted, target):
    target_vector = np.zeros_like(predicted)
    target_vector[target] = 1
    return ((predicted - target_vector) ** 2)

In [186]:
def mse_loss_derivative(predicted, target):
    target_vector = np.zeros_like(predicted)
    target_vector[target] = 1
    return 2 * (predicted - target_vector)

In [187]:
def dRelu(input):
    return np.where(input > 0, 1, 0)

In [188]:
def gradient_descent(learning_rate,w1,w2,w3,b1,b2,epochs,i=0):
    #global train_images , train_labels
    index = np.random.randint(0,train_images.shape[0])
    input = train_images[index]
    target = train_labels[index]
    
    
    predicted = model(input)
    output_1 = w1 @ input + b1
    output_1 = np.maximum(0, output_1)
    output_2 = w2 @ output_1 + b2   # Pre-activation output of the second layer
    output_2 = np.maximum(0, output_2) 
    output_final = output_2 @ w3
    output_final = np.maximum(0, output_final) / np.max(output_final)
    loss = loss_function(predicted, target)
    dL_dOutput = mse_loss_derivative(predicted, target)
    dOutput_w3 =  output_2  # This is the input to w3
    
    dL_dw3 =   dOutput_w3.T @ np.multiply(dL_dOutput,dRelu(output_final))
    
 
    dL_dw2 = np.multiply(np.multiply(dL_dOutput,dRelu(output_final)) @ w3.T,dRelu(output_2)) @ output_1.T
    
    
    dL_db2 = np.multiply(np.multiply(dL_dOutput,dRelu(output_final)) @ w3.T,dRelu(output_2))
    
    dL_dw1 = np.multiply(w2.T @ np.multiply(np.multiply(dL_dOutput,dRelu(output_final)) @ w3.T,dRelu(output_2)),dRelu(output_1)) @ input.T

    dL_db1 = np.multiply(w2.T @ np.multiply(np.multiply(dL_dOutput,dRelu(output_final)) @ w3.T,dRelu(output_2)),dRelu(output_1))
    w1 -= learning_rate * dL_dw1
    b1 -= learning_rate * dL_db1
    w2 -= learning_rate * dL_dw2
    b2 -= learning_rate * dL_db2
    w3 -= learning_rate * dL_dw3
    if i % 100 == 0:
        print(f'Epoch: {i}, Loss: {np.mean(loss)}')
        #print(np.sum(dL_dw1))
    if i == epochs:    
        return loss
    return gradient_descent(learning_rate,w1,w2,w3,b1,b2,epochs,i+1)
    
    

In [218]:
gradient_descent( 0.001,w1,w2,w3,b1,b2,2000)

Epoch: 0, Loss: 0.07972494821246891
Epoch: 100, Loss: 0.00808670511502296
Epoch: 200, Loss: 0.015394136963502498
Epoch: 300, Loss: 0.017955660289404974
Epoch: 400, Loss: 0.09635799173543072
Epoch: 500, Loss: 0.056493021223456674
Epoch: 600, Loss: 0.021262730707381793
Epoch: 700, Loss: 0.012870086890949367
Epoch: 800, Loss: 0.03429554243335863
Epoch: 900, Loss: 0.028083025225454288
Epoch: 1000, Loss: 0.009488265015589652
Epoch: 1100, Loss: 0.09529957588519872
Epoch: 1200, Loss: 0.01674858615690943
Epoch: 1300, Loss: 0.08025750547713309
Epoch: 1400, Loss: 0.045704668243095485
Epoch: 1500, Loss: 0.03933045468662966
Epoch: 1600, Loss: 0.05849630812907973
Epoch: 1700, Loss: 0.15688796320463755
Epoch: 1800, Loss: 0.05704453374040334
Epoch: 1900, Loss: 0.06719794497998452
Epoch: 2000, Loss: 0.035575339260077894


array([[5.86567673e-04],
       [6.61560669e-03],
       [1.84569585e-02],
       [1.64813558e-03],
       [0.00000000e+00],
       [1.36300215e-03],
       [2.91598964e-04],
       [7.53225894e-07],
       [3.26570666e-01],
       [2.20104038e-04]])

In [221]:
## Test
rights = 0
for i in range(1000):
    k = np.random.randint(len(test_images))
    predicted = model(test_images[k])
    
    if(np.argmax(predicted) == test_labels[k]):
        rights += 1
print("Accuracy: ", rights/10)

Accuracy:  66.8
